# MCPResource in OpenDXA

This tutorial covers the Model Context Protocol (MCP) resource in OpenDXA, which provides a standardized way to integrate external services and tools into your agents.

## Learning Objectives

By the end of this tutorial, you will understand:

1. What MCP is and how it works
2. How to create and use MCP resources
3. How to work with different transport types (STDIO and HTTP)
4. How to discover and use MCP tools
5. Best practices for MCP resource usage

## Prerequisites

- Basic understanding of OpenDXA's architecture
- Familiarity with Python async/await syntax
- Understanding of basic resource management concepts

## 1. Understanding MCP

The Model Context Protocol (MCP) is a standardized way to expose data and functionality to LLM applications. MCP servers can:

1. **Expose Data**: Through resources (similar to GET endpoints)
2. **Provide Functionality**: Through tools (similar to POST endpoints)
3. **Define Interaction Patterns**: Through prompts (reusable templates)

Let's start by creating a simple MCP resource. First let’s install the required packages:

In [ ]:
!npm install @modelcontextprotocol/server-filesystem
!npm install @modelcontextprotocol/server-brave-search
!npm install @modelcontextprotocol/server-sequential-thinking

In [ ]:
from opendxa import McpResource, ReasoningStrategy, DXA_LOGGER
from pprint import pprint


# From dictionary
dirname = "/Users/ctn/"
filesystem_resource = McpResource.from_config("filesystem", {
    "command": "npx",
    "args": ["-y", "@modelcontextprotocol/server-filesystem", dirname]
})
pprint(await filesystem_resource.list_tools())

# search_resource = McpResource.from_config("brave-search", {
#    "command": "npx",
#    "args": ["-y", "@modelcontextprotocol/server-brave-search"]
# })
# pprint(await search_resource.list_tools())

sequential_thinking_resource = McpResource.from_config("sequential-thinking", {
    "command": "npx",
    "args": ["-y", "@modelcontextprotocol/server-sequential-thinking"]
})
pprint(await sequential_thinking_resource.list_tools())

# response = await filesystem_resource.query({
#    "tool": "list_allowed_directories",
#    "arguments": {}  # {"path": dirname}
# })
# pprint(f"Resource response: {response}")

# We can then make that resource available to an Agent.
from opendxa import Agent, DXA_LOGGER

agent = Agent()\
    .with_model("anthropic:claude-3-sonnet-20240229")\
    .with_model("deepseek:deepseek-coder")\
    .with_resources({"filesystem": filesystem_resource})\
    .with_resources({"sequential-thinking": sequential_thinking_resource})\
    .with_reasoning(ReasoningStrategy.CHAIN_OF_THOUGHT)
DXA_LOGGER.basicConfig(level=DXA_LOGGER.DEBUG)
result = agent.ask("Can you reason out what I do for a living from my files? You can use ONE level of subdirs.")

pprint(result["choices"][0].message.content)

[Tool(name='read_file', description='Read the complete contents of a file from the file system. Handles various text encodings and provides detailed error messages if the file cannot be read. Use this tool when you need to examine the contents of a single file. Only works within allowed directories.', inputSchema={'type': 'object', 'properties': {'path': {'type': 'string'}}, 'required': ['path'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}),
 Tool(name='read_multiple_files', description="Read the contents of multiple files simultaneously. This is more efficient than reading files one by one when you need to analyze or compare multiple files. Each file's content is returned with its path as a reference. Failed reads for individual files won't stop the entire operation. Only works within allowed directories.", inputSchema={'type': 'object', 'properties': {'paths': {'type': 'array', 'items': {'type': 'string'}}}, 'required': ['paths'], 'additionalP

22:35:25 - [opendxa.common.resource.mcp.mcp_resource.McpResource] ERROR - Tool listing failed: unhandled errors in a TaskGroup (1 sub-exception)
  + Exception Group Traceback (most recent call last):
  |   File "/Users/ctn/src/aitomatic/opendxa/opendxa/common/resource/mcp/mcp_resource.py", line 397, in list_tools
  |     async with stdio_client(self.server_params) as (read, write):
  |                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  |   File "/Users/ctn/.pyenv/versions/3.12.7/lib/python3.12/contextlib.py", line 231, in __aexit__
  |     await self.gen.athrow(value)
  |   File "/Users/ctn/src/aitomatic/opendxa/.venv/lib/python3.12/site-packages/mcp/client/stdio/__init__.py", line 167, in stdio_client
  |     anyio.create_task_group() as tg,
  |     ^^^^^^^^^^^^^^^^^^^^^^^^^
  |   File "/Users/ctn/src/aitomatic/opendxa/.venv/lib/python3.12/site-packages/anyio/_backends/_asyncio.py", line 772, in __aexit__
  |     raise BaseExceptionGroup(
  | ExceptionGroup: unhandled errors in a TaskG

[]


22:35:27 - [opendxa.common.resource.llm_resource.LLMResource] INFO - REQUEST TO deepseek:deepseek-coder:
--------------------------------------------------------------------------------
[{'role': 'system', 'content': "You are executing a reasoning task. Provide clear, logical analysis and reasoning.\nYou are operating within a three-layer execution hierarchy: Workflow -> Plan -> Reasoning.\nThe Workflow layer is typically specified by the human operator\nThe Plan layer is typically generated dynamically to accomplish the objective\nThe Reasoning layer is typically a choice of several fundamental strategies, e.g., chain-of-thought, tree-of-thought, reflection, OODA loop, etc.\n\nYou are currently in the Reasoning layer. Execute the reasoning task while keeping in mind:\n 1. The broader workflow context and its objectives\n 2. The specific plan that this reasoning task is part of\n 3. The immediate reasoning task requirements\n\nHere is your reasoning strategy:\nReasoning: Following chai

[04/13/25 22:35:28] INFO     HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 ]8;id=974629;file:///Users/ctn/src/aitomatic/opendxa/.venv/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=139747;file:///Users/ctn/src/aitomatic/opendxa/.venv/lib/python3.12/site-packages/httpx/_client.py#1038\1038]8;;\
                             200 OK"                                                                               

...

[04/13/25 22:35:52] INFO     HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 ]8;id=449181;file:///Users/ctn/src/aitomatic/opendxa/.venv/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=852908;file:///Users/ctn/src/aitomatic/opendxa/.venv/lib/python3.12/site-packages/httpx/_client.py#1038\1038]8;;\
                             200 OK"                                                                               

22:35:56 - [opendxa.common.resource.llm_resource.LLMResource] INFO - RESPONSE FROM deepseek:deepseek-coder:
--------------------------------------------------------------------------------
ChatCompletion(id='b589f4b0-ca71-498d-a026-a76d86e45f73', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_0_0f928a04-ded9-49a7-a698-2993a3f3b32e', function=Function(arguments='{"path":"/Users/ctn"}', name='filesystem__a45531ec__list_directory'), type='function', index=0)]))], created=1744558551, model='deepseek-chat', object='chat.completion', service_tier=None, system_fingerprint='fp_3d5141a69a_prod0225', usage=CompletionUsage(completion_tokens=30, prompt_tokens=3968, total_tokens=3998, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=3904), prompt_ca

[04/13/25 22:35:59] INFO     HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 ]8;id=924254;file:///Users/ctn/src/aitomatic/opendxa/.venv/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=324251;file:///Users/ctn/src/aitomatic/opendxa/.venv/lib/python3.12/site-packages/httpx/_client.py#1038\1038]8;;\
                             200 OK"                                                                               

22:36:32 - [opendxa.common.resource.llm_resource.LLMResource] INFO - RESPONSE FROM deepseek:deepseek-coder:
--------------------------------------------------------------------------------
ChatCompletion(id='8303791e-285a-4b0b-97ed-ad7614266253', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="From the directory listing, here's the analysis to infer your profession:\n\n### Step 1: Key Observations\n1. **Technical Files and Directories**:\n   - `.bashrc`, `.bash_profile`, `.gitconfig`, `.vimrc`, `.zshrc`: These are configuration files for development tools like Bash, Git, Vim, and Zsh, indicating familiarity with programming and system administration.\n   - `PycharmProjects`, `.vscode`, `src`, `tests`, `node_modules`, `package.json`, `requirements.txt`: These suggest active involvement in software development, possibly using Python (Pycharm, requirements.txt) and JavaScript/Node.js (node_modules, package.json).\n   - `HFAI Framework D

("From the directory listing, here's the analysis to infer your profession:\n"
 '\n'
 '### Step 1: Key Observations\n'
 '1. **Technical Files and Directories**:\n'
 '   - `.bashrc`, `.bash_profile`, `.gitconfig`, `.vimrc`, `.zshrc`: These are '
 'configuration files for development tools like Bash, Git, Vim, and Zsh, '
 'indicating familiarity with programming and system administration.\n'
 '   - `PycharmProjects`, `.vscode`, `src`, `tests`, `node_modules`, '
 '`package.json`, `requirements.txt`: These suggest active involvement in '
 'software development, possibly using Python (Pycharm, requirements.txt) and '
 'JavaScript/Node.js (node_modules, package.json).\n'
 '   - `HFAI Framework Design Testing.ipynb`: A Jupyter Notebook file, often '
 'used in data science, machine learning, or research.\n'
 '\n'
 '2. **Professional Documents**:\n'
 '   - `190911 FUV Follow-Ups.docx`, `Cathay-KIX-HAN-2.pdf`, '
 '`2019-08-31-Panasonic-Kaiser.pdf`: These files hint at professional '
 'communicat

## 2. Launching our own Python MCP services

We can create our own Python MCP services and launch them automatically via an `MCPResource` that is attached to those services.

In [ ]:
from opendxa.common.resource.mcp import HttpTransportParams

# Example 1: STDIO Transport (Local Server)
local_mcp = McpResource(
    name="local_echo",
    transport_params=StdioTransportParams(
        server_script="examples/learning_paths/02_core_concepts/mcp_servers/mcp_echo.py",
        command="python",
        args=["examples/learning_paths/02_core_concepts/mcp_servers/mcp_echo.py"],
        env={"DEBUG": "1"}  # Optional environment variables
    )
)

# Example 2: HTTP Transport (Remote Server)
remote_mcp = McpResource(
    name="remote_echo",
    transport_params=HttpTransportParams(
        url="https://api.example.com/mcp",
        headers={"Authorization": "Bearer your-token"},
        timeout=5.0,  # Connection timeout in seconds
        sse_read_timeout=300.0  # SSE read timeout in seconds
    )
)

# Test both servers
local_response = await local_mcp.query({
    "tool": "ping"
})
print(f"Local server response: {local_response.content}")

try:
    remote_response = await remote_mcp.query({
        "tool": "ping"
    })
    print(f"Remote server response: {remote_response.content}")
except Exception as e:
    print(f"Remote server error: {e}")

## 3. Tool Discovery and Usage

MCP resources provide a way to discover available tools at runtime. Let's see how to use this feature:

In [ ]:
# Discover available tools
tools = await local_mcp.list_tools()
print(f"Found {len(tools)} available tools\n")

# Print tool details
for tool in tools:
    print(f"Tool: {tool.name}")
    print(f"Description: {tool.description}")
    print("Parameters:")
    for param_name, param_details in tool.inputSchema["properties"].items():
        print(f"  - {param_name}: {param_details.get('type')}")
        if param_name in tool.inputSchema.get("required", []):
            print("    (Required)")
    print()

# Example: Using a discovered tool
if tools:
    tool = tools[0]  # Use the first available tool
    print(f"Testing tool: {tool.name}")

    # Prepare arguments based on the tool's schema
    arguments = {}
    for param_name, param_details in tool.inputSchema["properties"].items():
        if param_name in tool.inputSchema.get("required", []):
            # Provide a default value based on the parameter type
            param_type = param_details.get("type")
            if param_type == "string":
                arguments[param_name] = "test"
            elif param_type == "number":
                arguments[param_name] = 42
            elif param_type == "boolean":
                arguments[param_name] = True
            elif param_type == "array":
                arguments[param_name] = []
            elif param_type == "object":
                arguments[param_name] = {}

    # Execute the tool
    response = await local_mcp.query({
        "tool": tool.name,
        "arguments": arguments
    })

    print(f"Tool response: {response.content}")

## 4. Error Handling

MCP resources provide robust error handling. Let's see how to handle different types of errors:

In [ ]:
# Example 1: Invalid tool name
try:
    response = await local_mcp.query({
        "tool": "nonexistent_tool",
        "arguments": {}
    })
    print(f"Response: {response.content}")
except Exception as e:
    print(f"Error: {e}")

# Example 2: Invalid arguments
try:
    response = await local_mcp.query({
        "tool": "echo",
        "arguments": {"invalid_param": "value"}
    })
    print(f"Response: {response.content}")
except Exception as e:
    print(f"Error: {e}")

# Example 3: Missing required arguments
try:
    response = await local_mcp.query({
        "tool": "echo"  # Missing required 'message' argument
    })
    print(f"Response: {response.content}")
except Exception as e:
    print(f"Error: {e}")

## 5. Advanced Features

### 5.1 Environment Variables

You can pass environment variables to local MCP servers:

In [ ]:
# Create MCP resource with environment variables
mcp_with_env = McpResource(
    name="env_mcp",
    transport_params=StdioTransportParams(
        server_script="examples/learning_paths/02_core_concepts/mcp_servers/mcp_echo.py",
        command="python",
        args=["examples/learning_paths/02_core_concepts/mcp_servers/mcp_echo.py"],
        env={
            "DEBUG": "1",
            "LOG_LEVEL": "INFO",
            "CUSTOM_VAR": "custom_value"
        }
    )
)

# Test the server with environment variables
response = await mcp_with_env.query({
    "tool": "ping"
})
print(f"Response: {response.content}")

### 5.2 Tool Schema Validation

MCP automatically validates tool arguments against their schemas:

In [ ]:
# Example 1: Valid arguments
try:
    response = await local_mcp.query({
        "tool": "echo",
        "arguments": {"message": "Valid message"}
    })
    print(f"Valid arguments response: {response.content}")
except Exception as e:
    print(f"Error: {e}")

# Example 2: Invalid argument type
try:
    response = await local_mcp.query({
        "tool": "echo",
        "arguments": {"message": 42}  # Should be a string
    })
    print(f"Response: {response.content}")
except Exception as e:
    print(f"Error: {e}")

## 6. Best Practices

Here are some best practices for working with MCP resources:

1. **Server Design**
   - Keep servers focused on specific functionality
   - Implement proper error handling
   - Use type hints and docstrings
   - Follow the single responsibility principle

2. **Client Usage**
   - Always initialize resources before use
   - Implement proper error handling
   - Use appropriate timeouts for remote servers
   - Validate tool arguments before calling

3. **Transport Selection**
   - Use STDIO transport for local servers
   - Use HTTP transport for remote servers
   - Configure appropriate timeouts
   - Handle connection errors gracefully

4. **Tool Design**
   - Keep tools focused and single-purpose
   - Provide clear documentation
   - Use appropriate parameter types
   - Handle edge cases

5. **Error Handling**
   - Implement proper error handling
   - Use appropriate error messages
   - Handle timeouts and connection errors
   - Validate inputs and outputs

## Summary

In this tutorial, we covered:

1. Understanding MCP and its features
2. Working with different transport types
3. Discovering and using MCP tools
4. Handling errors and edge cases
5. Using advanced features
6. Following best practices

The MCP resource in OpenDXA provides a powerful and flexible way to integrate external services and tools into your agents. By following the best practices outlined in this tutorial, you can create robust and maintainable MCP-based applications.